In [1]:
"""Test ChatGLM"""
import random
import unittest
import numpy as np

import mindspore
from mindspore import Tensor

from mindnlp.models.glm.chatglm import ChatGLMForConditionalGeneration
from mindnlp.transforms.tokenizers import ChatGLMTokenizer

/home/daiyuxin/anaconda3/envs/kewei-mindspore/lib/python3.9/site-packages/mindnlp/utils/download.py:29: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
def set_random_seed(seed):
    """set random seed"""
    random.seed(seed)

    # mindspore RNGs
    mindspore.set_seed(seed)

    # numpy RNG
    np.random.seed(seed)


def ids_tensor(shape, vocab_size):
    """Creates a random int32 tensor of the shape within the vocab size"""
    total_dims = 1
    for dim in shape:
        total_dims *= dim

    values = []
    for _ in range(total_dims):
        values.append(random.randint(0, vocab_size - 1))

    return mindspore.Tensor(values, dtype=mindspore.int64).view(shape)


def get_model_and_tokenizer():
    """get model and tokenizer"""
    model = ChatGLMForConditionalGeneration.from_pretrained("chatglm-6b")

    tokenizer = ChatGLMTokenizer.from_pretrained("chatglm-6b")
    return model, tokenizer

In [3]:
model = ChatGLMForConditionalGeneration.from_pretrained("chatglm-6b")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

[WARNING] ME(1883886:140633567283008,MainProcess):2023-09-19-20:55:36.959.960 [mindspore/lib/python3.9/site-packages/mindnlp/abc/models/pretrained_model.py:457] The following parameters in checkpoint files are not loaded:
['transformer.layers.0.attention.rotary_emb.inv_freq', 'transformer.layers.1.attention.rotary_emb.inv_freq', 'transformer.layers.2.attention.rotary_emb.inv_freq', 'transformer.layers.3.attention.rotary_emb.inv_freq', 'transformer.layers.4.attention.rotary_emb.inv_freq', 'transformer.layers.5.attention.rotary_emb.inv_freq', 'transformer.layers.6.attention.rotary_emb.inv_freq', 'transformer.layers.10.attention.rotary_emb.inv_freq', 'transformer.layers.11.attention.rotary_emb.inv_freq', 'transformer.layers.7.attention.rotary_emb.inv_freq', 'transformer.layers.8.attention.rotary_emb.inv_freq', 'transformer.layers.9.attention.rotary_emb.inv_freq', 'transformer.layers.12.attention.rotary_emb.inv_freq', 'transformer.layers.13.attention.rotary_emb.inv_freq', 'transformer.laye

In [4]:
# while True:
#     try:
#         model = ChatGLMForConditionalGeneration.from_pretrained("chatglm-6b")
#         break
#     except:
#         continue

In [7]:
?tokenizer

Signature:      tokenizer(text_input)
Type:           ChatGLMTokenizer
String form:    <mindnlp.transforms.tokenizers.chatglm_tokenizer.ChatGLMTokenizer object at 0x7fe6cc365130>
File:           ~/anaconda3/envs/kewei-mindspore/lib/python3.9/site-packages/mindnlp/transforms/tokenizers/chatglm_tokenizer.py
Docstring:     
Construct a ChatGLM tokenizer. Based on byte-level Byte-Pair-Encoding.

Args:
    vocab_file (`str`):
        Path to the vocabulary file.
Call docstring: Call method for input conversion for eager mode with C++ implementation.

In [5]:
tokenizer = ChatGLMTokenizer.from_pretrained("chatglm-6b")

In [9]:
??model.stream_chat

Signature:
model.stream_chat(
    tokenizer,
    query: str,
    history: List[Tuple[str, str]] = None,
    max_length: int = 2048,
    do_sample=True,
    top_p=0.7,
    temperature=0.95,
    logits_processor=None,
    **kwargs,
)
Source:   
    def stream_chat(self, tokenizer, query: str, history: List[Tuple[str, str]] = None, max_length: int = 2048,
                    do_sample=True, top_p=0.7, temperature=0.95, logits_processor=None, **kwargs):
        """stream chat"""
        if history is None:
            history = []
        if logits_processor is None:
            logits_processor = LogitsProcessorList()
        logits_processor.append(InvalidScoreLogitsProcessor())
        gen_kwargs = {"max_length": max_length, "do_sample": do_sample, "top_p": top_p,
                      "temperature": temperature, "logits_processor": logits_processor, **kwargs}
        if not history:
            prompt = query
        else:
            prompt = ""
            for i, (old_query, response

In [14]:
from mindnlp.models.glm.chatglm import LogitsProcessorList,InvalidScoreLogitsProcessor

In [17]:
def stream_chat(tokenizer, query: str, history= None, max_length: int = 2048,
                do_sample=True, top_p=0.7, temperature=0.95, logits_processor=None, **kwargs):
    """stream chat"""
    if history is None:
        history = []
    if logits_processor is None:
        logits_processor = LogitsProcessorList()
    logits_processor.append(InvalidScoreLogitsProcessor())
    gen_kwargs = {"max_length": max_length, "do_sample": do_sample, "top_p": top_p,
                  "temperature": temperature, "logits_processor": logits_processor, **kwargs}
    if not history:
        prompt = query
    else:
        prompt = ""
        for i, (old_query, response) in enumerate(history):
            prompt += "[Round {}]\n问：{}\n答：{}\n".format(i, old_query, response)
        prompt += "[Round {}]\n问：{}\n答：".format(len(history), query)
    inputs = tokenizer(prompt)
    for outputs in model.stream_generate(**inputs, **gen_kwargs):
        outputs = outputs.tolist()[0][len(inputs["input_ids"][0]):]
        response = tokenizer.decode(outputs)
        response = model.process_response(response)
        new_history = history + [(query, response)]
        yield response, new_history

In [32]:
import mindspore as ms

In [33]:
def chat(tokenizer, query: str, history = None, max_length: int = 2048, num_beams=1,
         do_sample=True, top_p=0.7, temperature=0.95, logits_processor=None, **kwargs):
    """chat."""
    if history is None:
        history = []
    if logits_processor is None:
        logits_processor = LogitsProcessorList()
    logits_processor.append(InvalidScoreLogitsProcessor())
    gen_kwargs = {"max_length": max_length, "num_beams": num_beams, "do_sample": do_sample, "top_p": top_p,
                  "temperature": temperature, "logits_processor": logits_processor, **kwargs}
    if not history:
        prompt = query
    else:
        prompt = ""
        for i, (old_query, response) in enumerate(history):
            prompt += "[Round {}]\n问：{}\n答：{}\n".format(i, old_query, response)
        prompt += "[Round {}]\n问：{}\n答：".format(len(history), query)
    inputs = tokenizer(prompt)
    outputs = model.generate(ms.Tensor(inputs), **gen_kwargs)
    outputs = outputs.tolist()[0][len(inputs["input_ids"][0]):]
    response = tokenizer.decode(outputs)
    response = model.process_response(response)
    history = history + [(query, response)]
    return response, history

In [35]:
parameters = [
            ("晚上睡不着怎么办", False, 2048, 1, True, 4),
            ("介绍一下清华大学", False, 64, 1, False, 1),
            ("推荐几个电影", False, 2048, 1, True, 4),
            ("怎么用Pytorch写一个模型？", False, 2048, 1, True, 4),
            #   (True, 2048, 1),
            #   (True, 64, 1),
            #   (True, 2048, 4)
              ]
for sentence, do_sample, max_length, num_beams, use_bucket, bucket_num in parameters:
    set_random_seed(42)
    inputs = tokenizer(sentence)
    inputs = Tensor([inputs])
    outputs = model.generate(
        inputs,
        do_sample=do_sample,
        max_length=max_length,
        num_beams=num_beams,
        jit=True,
        use_bucket=use_bucket,
        bucket_num=bucket_num
    )

    outputs = outputs.asnumpy().tolist()[0]
    out_sentence = tokenizer.decode(outputs, skip_special_tokens=True)
    print(out_sentence)

晚上睡不着怎么办 以下是一些可能有助于改善晚上睡不着的方法:

1. 保持规律的睡眠时间:尽量在同一时间入睡和起床,即使在周末和假期也要保持一致。

2. 创造舒适的睡眠环境:保持房间安静、凉爽、黑暗和舒适。使用舒适的床垫和枕头,并确保房间的温度适宜。

3. 放松身体和思维:在睡前进行一些放松活动,如冥想、深呼吸、瑜伽或渐进性肌肉松弛练习。避免使用电子设备,因为它们会刺激大脑,使你更加清醒。

4. 避免饮食和饮料:在睡前几个小时内避免饮用咖啡、茶、可乐和其他含咖啡因的饮料。避免吃辛辣、油腻或难以消化的食物。

5. 避免过度兴奋:在睡前几个小时内避免进行过度兴奋的活动,如看电视、玩游戏或工作。

6. 尝试使用睡眠辅助工具:如果以上方法仍然无法帮助入睡,可以考虑使用睡眠辅助工具,如白噪音、振动器或睡眠面罩。

如果这些方法无效,建议咨询医生或睡眠专家,以获取更专业的建议和帮助。
介绍一下清华大学 清华大学是中国著名的综合性大学,位于北京市海淀区,创建于1911年,前身是清华学堂。学校被誉为“中国科学的摇篮”,是中国政府和企业的重要人才培养基地之一。

清华大学在工程、科学、经济、管理、文学、艺术、哲学、法学等领域都拥有
推荐几个电影 以下都是不同类型的电影,可以根据自己的喜好选择观看:

1.《肖申克的救赎》(The Shawshank Redemption):一部经典的剧情片,讲述了一个银行家在肖申克监狱中的故事,讲述了希望、友谊和自由的重要性。

2.《泰坦尼克号》(Titanic):一部浪漫史诗片,讲述了一对来自不同社会阶层的情侣在泰坦尼克号号船上的故事,讲述了爱情、勇气和生命的重要性。

3.《阿甘正传》(Forrest Gump):一部经典的剧情片,讲述了一个智商不高但乐观向上的男人在生命中的故事,讲述了人生的意义、爱、友谊和生命的重要性。

4.《星际穿越》(Interstellar):一部科幻片,讲述了一组宇航员在宇宙中探索新星系的故事,讲述了人类的生存、希望和勇气。

5.《少年派的奇幻漂流》(Life of Pi):一部冒险片,讲述了一个印度少年在海上漂流中的故事,讲述了信仰、友谊和生命的重要性。
怎么用Pytorch写一个模型? 

PyTorch 是一个强大的深度学习框架,它提供了许多方便的工具和函数,使得编写深度学习模型变得更加容易。下面

In [53]:
%pip install uvicorn

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/79/96/b0882a1c3f7ef3dd86879e041212ae5b62b4bd352320889231cc735a8e8f/uvicorn-0.23.2-py3-none-any.whl (59 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl (97 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/95/04/ff642e65ad6b90db43e668d70ffb6736436c7ce41fcc549f4e9472234127/h11-0.14.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [39]:
from fastapi import FastAPI, Request

In [ ]:
app = FastAPI()


@app.post("/")

In [50]:
import json, datetime

In [51]:
async def create_item(request: Request):
    global model, tokenizer
    json_post_raw = await request.json()
    json_post = json.dumps(json_post_raw)
    json_post_list = json.loads(json_post)
    prompt = json_post_list.get('prompt')
    history = json_post_list.get('history')
    max_length = json_post_list.get('max_length')
    top_p = json_post_list.get('top_p')
    temperature = json_post_list.get('temperature')
    set_random_seed(42)
    inputs = tokenizer(prompt)
    inputs = Tensor([inputs])
    outputs = model.generate(
        inputs,
        do_sample=do_sample,
        max_length=max_length,
        num_beams=num_beams,
        jit=True,
        use_bucket=use_bucket,
        bucket_num=bucket_num
    )

    outputs = outputs.asnumpy().tolist()[0]
    out_sentence = tokenizer.decode(outputs, skip_special_tokens=True)
    
    now = datetime.datetime.now()
    time = now.strftime("%Y-%m-%d %H:%M:%S")
    answer = {
        "response": out_sentence,
        "history": history,
        "status": 200,
        "time": time
    }
    log = "[" + time + "] " + '", prompt:"' + prompt + '", response:"' + repr(out_sentence) + '"'
    print(log)
    return answer

In [52]:
# # 调用函数示例
# request_data = {
#     "prompt": "你好",
#     "history": [],
#     "max_length": 1024,
#     "top_p": 0.7,
#     "temperature": 0.95
# }

# response = await create_item(request_data)
# print(response)

[2023-09-19 23:00:59] ", prompt:"你好", response:"'你好 你好👋!我是人工智能助手 ChatGLM-6B,很高兴见到你,欢迎问我任何问题。'"
{'response': '你好 你好👋!我是人工智能助手 ChatGLM-6B,很高兴见到你,欢迎问我任何问题。', 'history': [], 'status': 200, 'time': '2023-09-19 23:00:59'}


In [ ]:
if __name__ == '__main__':
    uvicorn.run(app, host='0.0.0.0', port=8000, workers=1)